In [1]:
import pyodbc
import os.path
import glob
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
import getpass

import warnings
warnings.simplefilter("ignore")

pd.set_option('display.max_columns', None)

In [2]:
# SQL python connection to our server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=sql-2-db;'
                      'Database=CBQ2;')
                    
cursor = conn.cursor()

In [3]:
# Code to get last sales diagnostic file from folder.
folder_path = r'G:\SALES\2019 Sales Reports\Sell Through Reporting\Ingram'
file_glob_detail = r'\*Ingram*xlsx'
files = glob.glob(folder_path + file_glob_detail)
file_detail = max(files, key=os.path.getctime)

# file_glob_summary = r'\*Summary*csv'
# files = glob.glob(folder_path + file_glob_summary)
# file_summary = max(files, key=os.path.getctime)

In [4]:
for f in files:
    print(f)

G:\SALES\2019 Sales Reports\Sell Through Reporting\Ingram\2019 Flash Ingram 010519 v 122918.xlsx
G:\SALES\2019 Sales Reports\Sell Through Reporting\Ingram\2019 Flash Ingram 011219 v 010519.xlsx
G:\SALES\2019 Sales Reports\Sell Through Reporting\Ingram\2019 Flash Ingram 011919 v 011219.xlsx
G:\SALES\2019 Sales Reports\Sell Through Reporting\Ingram\2019 Flash Ingram 020219 v 012619.xlsx
G:\SALES\2019 Sales Reports\Sell Through Reporting\Ingram\2019 Flash Ingram 020919 v 020219.xlsx
G:\SALES\2019 Sales Reports\Sell Through Reporting\Ingram\2019 Flash Ingram 022319 v 021619.xlsx
G:\SALES\2019 Sales Reports\Sell Through Reporting\Ingram\2019 Flash Ingram 030219 v 022319.xlsx
G:\SALES\2019 Sales Reports\Sell Through Reporting\Ingram\2019 Flash Ingram 101319 - 101519.xlsx
G:\SALES\2019 Sales Reports\Sell Through Reporting\Ingram\2019 Flash Ingram 101619 - 101719.xlsx
G:\SALES\2019 Sales Reports\Sell Through Reporting\Ingram\2019 Flash Ingram 101819 - 102019.xlsx
G:\SALES\2019 Sales Reports\Se

In [5]:
type(files)

list

In [17]:
files_q4 = files[7:]

In [93]:
df_new = pd.DataFrame()

for i,f in enumerate(files_q4):
    df_week = pd.read_excel(f
                   ,usecols=[0,4,11]
                   ,engine='openpyxl'
                   ,skiprows=1
                    ,dtype={'ISBN 13':str},
                   )
    df_new = pd.concat([df_new,df_week])
    print(f"Rows added: {df_new.shape[0]:,}.\t ||| \tPercent Completed: {(i+1)/len(files_q4):.0%}")
    print(df_week.head(2))
    print()

Rows added: 6,481.	 ||| 	Percent Completed: 4%
         PUB        ISBN 13  Sales (10/13 - 10/15)
0  Chronicle  9781452136448                    125
1  Chronicle  9781452182759                     98

Rows added: 13,142.	 ||| 	Percent Completed: 7%
          PUB        ISBN 13  Sales (10/16 - 10/17)
0   Chronicle  9781452175218                     95
1  Tourbillon  9782747086998                     66

Rows added: 19,739.	 ||| 	Percent Completed: 11%
         PUB        ISBN 13  Sales (10/18 - 10/20)
0  Moleskine  8051272892741                     64
1  Chronicle  9781452128306                     50

Rows added: 26,231.	 ||| 	Percent Completed: 14%
         PUB        ISBN 13  Sales (10/21 - 10/22)
0  Chronicle  9781452182759                    102
1  Chronicle  9781452111735                     85

Rows added: 32,693.	 ||| 	Percent Completed: 18%
         PUB        ISBN 13  Sales (10/23 - 10/24)
0  Chronicle  9781452182759                   3261
1  Chronicle  9781452171197          

In [94]:
df_new.dtypes

PUB                                   object
ISBN 13                               object
Sales (10/13 - 10/15)                float64
Sales (10/16 - 10/17)                float64
Sales (10/18 - 10/20)                float64
Sales (10/21 - 10/22)                float64
Sales (10/23 - 10/24)                float64
Sales (10/25 - 10/27)                float64
Sales (10/28 - 10/29)                float64
Sales (10/30 - 10/31)                float64
Sales (10/31 - 11/07)                float64
Sales (11/01 - 11/03)                float64
Sales (11/03 - 11/10)                float64
Sales (11/05 - 11/12)                float64
Sales (11/07 - 11/14)                float64
Sales (11/10 - 11/17)                float64
Sales (11/12 - 11/19)                float64
Sales (11/14 - 11/21)                float64
Sales (11/17 - 11/24)                float64
Sales (11/19 - 11/26)                float64
Sales (11/21 - 11/28)                float64
Sales (11/24 - 12/01)                float64
Sales (11/

In [95]:
df_new.drop(["This Week's Sales"],axis=1,inplace=True)

In [100]:
df_new = df_new.melt(id_vars=['PUB','ISBN 13']
            ,var_name = 'Date'
            ,value_name = 'Value')

In [107]:
df_new = df_new.loc[~df_new['PUB'].isnull()]

In [108]:
df_new.isnull().sum()

PUB              0
ISBN 13          0
Date             0
Value      4877256
dtype: int64

In [109]:
df_new.shape

(5058180, 4)

In [122]:
df_pub_date = df_new.groupby(['PUB','Date']).sum().reset_index()

In [129]:
df_pub_date['PUB'].unique()

array(['-', 'Chronicle', 'Creative Company', 'Do Books', 'Galison',
       'Hardie Grant Publishing', 'Laurence King', 'Moleskine',
       'PQ Blackwell', 'Princeton', 'Sierra Club', 'Tourbillon'],
      dtype=object)

In [134]:
df_pub_date = df_pub_date.loc[~(df_pub_date['PUB']== "-")]

In [139]:
df_pub_date.head(20)

,PUB,Date,Value
27,Chronicle,Sales (10/13 - 10/15),4850.0
28,Chronicle,Sales (10/16 - 10/17),3703.0
29,Chronicle,Sales (10/18 - 10/20),1612.0
30,Chronicle,Sales (10/21 - 10/22),4329.0
31,Chronicle,Sales (10/23 - 10/24),6907.0
32,Chronicle,Sales (10/25 - 10/27),2528.0
33,Chronicle,Sales (10/28 - 10/29),5019.0
34,Chronicle,Sales (10/30 - 10/31),3920.0
35,Chronicle,Sales (10/31 - 11/07),13102.0
36,Chronicle,Sales (11/01 - 11/03),1504.0


In [140]:
df_pub = df_pub_date.groupby('PUB').mean()

In [144]:
path = r'G:\SALES\2022 Sales Reports\Reports\Misc\2022.09.28_ingram_py.xlsx'
writer = pd.ExcelWriter(path, engine='xlsxwriter')

df_pub_date.to_excel(writer,sheet_name='pub_date_value',index = False)
df_pub.to_excel(writer,sheet_name='pub_value',index = True)

writer.save()